# Shor's Algorithm: Complete Implementation with GUI

This notebook provides a comprehensive implementation of Shor's algorithm with:
- Interactive GUI interface
- Quantum circuit visualization
- Probability analysis
- Step-by-step explanations
- Educational content about quantum computing concepts


In [ ]:
# Import all necessary libraries
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from fractions import Fraction
from math import gcd
from IPython.display import display, HTML, clear_output
import warnings
warnings.filterwarnings('ignore')

# Qiskit imports
from qiskit import QuantumCircuit, transpile
from qiskit.circuit.library import QFTGate
from qiskit.circuit import Gate
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram, circuit_drawer
from qiskit.quantum_info import Statevector

# Set up matplotlib for better plots
try:
    plt.style.use('seaborn-v0_8')
except:
    plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10


In [ ]:
# Create a simple interactive interface without ipywidgets
def create_shor_interface():
    """Create a simple interface for Shor's algorithm"""
    
    print("🔬 Shor's Algorithm Interactive Demo")
    print("=" * 50)
    print("Quantum Factorization with Educational Content")
    print("=" * 50)
    
    # Initialize Shor algorithm instance
    shor = ShorAlgorithm()
    
    return shor

def run_shor_demo():
    """Run a demonstration of Shor's algorithm"""
    shor = create_shor_interface()
    
    # Demo parameters
    N = 15
    n_count = 6
    shots = 1024
    tries = 5
    
    print(f"🚀 Running Shor's Algorithm to factor N = {N}")
    print("=" * 50)
    
    factors = shor.shors_algorithm(
        N=N,
        n_count=n_count,
        shots=shots,
        tries=tries,
        show_circuit=True
    )
    
    if factors is None:
        print(f"\n❌ Failed to find factors of {N} after {tries} attempts.")
        print("Try increasing the number of counting qubits or shots.")
    else:
        print(f"\n✅ Success! Found factors of {N}: {factors[0]} × {factors[1]} = {factors[0] * factors[1]}")
        print(f"\nVerification: {factors[0]} × {factors[1]} = {factors[0] * factors[1]}")
    
    return shor

def show_circuit_demo():
    """Show circuit visualization"""
    shor = ShorAlgorithm()
    print("🔬 Quantum Circuit Visualization")
    print("=" * 50)
    shor.display_circuit()

def show_probabilities_demo():
    """Show probability analysis"""
    shor = ShorAlgorithm()
    print("📊 Probability Analysis")
    print("=" * 50)
    shor.display_probabilities()

def show_steps_demo():
    """Show step-by-step guide"""
    shor = ShorAlgorithm()
    print("📚 Educational Content: Step-by-Step Guide")
    print("=" * 50)
    shor.display_step_by_step()



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Educational Content: Understanding Shor's Algorithm

### What is Shor's Algorithm?
Shor's algorithm is a quantum algorithm for integer factorization. It can factor large integers exponentially faster than classical algorithms, which has significant implications for cryptography.

### Key Quantum Computing Concepts

#### 1. Quantum Superposition
In quantum computing, a qubit can exist in a superposition of states |0⟩ and |1⟩:
|ψ⟩ = α|0⟩ + β|1⟩

This allows quantum computers to process multiple states simultaneously.

#### 2. Quantum Fourier Transform (QFT)
The QFT is a quantum version of the classical Fourier transform. It transforms quantum states from the computational basis to the frequency basis:

QFT|j⟩ = (1/√N) Σ(k=0 to N-1) e^(2πijk/N) |k⟩

**Role in Shor's Algorithm:** QFT is used to extract period information from quantum states, which is crucial for finding the period of modular exponentiation.

#### 3. Quantum Phase Estimation (QPE)
QPE is used to estimate the eigenvalue of a unitary operator. In Shor's algorithm, it's used to find the period of the function f(x) = a^x mod N.

### Shor's Algorithm Steps
1. **Classical preprocessing**: Check if N is even or has small factors
2. **Random selection**: Choose a random integer a coprime to N
3. **Order finding**: Use quantum phase estimation to find the period r of f(x) = a^x mod N
4. **Classical postprocessing**: Use the period to find factors of N


In [ ]:
class ShorAlgorithm:
    """Complete implementation of Shor's algorithm with GUI and educational features"""
    
    def __init__(self):
        self.backend = AerSimulator()
        self.current_circuit = None
        self.measurement_results = None
        self.step_explanations = []
        
    def modular_multiplication_unitary(self, a, N, n_target):
        """Create unitary matrix for modular multiplication by 'a' modulo N"""
        dim = 2 ** n_target
        M = np.zeros((dim, dim), dtype=complex)
        
        for x in range(dim):
            if x < N:
                y = (a * x) % N
                M[y, x] = 1.0
            else:
                M[x, x] = 1.0
        return M
    
    def continued_fraction_denominator(self, phase, max_denominator):
        """Extract denominator from phase using continued fractions"""
        frac = Fraction(phase).limit_denominator(max_denominator)
        return frac.denominator, frac.numerator
    
    def find_order_qpe(self, a, N, n_count=8, shots=1024, show_circuit=True):
        """Find the order of 'a' modulo N using Quantum Phase Estimation"""
        if gcd(a, N) != 1:
            return None, None, None
        
        n_target = max(1, math.ceil(math.log2(N)))
        
        # Create quantum circuit
        qc = QuantumCircuit(n_count + n_target, n_count)
        
        # Step 1: Initialize counting qubits in superposition
        qc.h(range(n_count))
        self.step_explanations.append(
            f"Step 1: Applied Hadamard gates to {n_count} counting qubits to create superposition state"
        )
        
        # Step 2: Initialize target register in |1⟩
        qc.x(n_count)
        self.step_explanations.append(
            f"Step 2: Initialized target register in |1⟩ state (|1⟩ = |00...01⟩)"
        )
        
        # Step 3: Apply controlled modular multiplication gates
        for j in range(n_count):
            exp = 2 ** j
            a_pow = pow(a, exp, N)
            M = self.modular_multiplication_unitary(a_pow, N, n_target)
            
            # Create controlled unitary gate
            u_gate = Gate(f"U^{exp}", 1 + n_target, [])
            u_gate.definition = QuantumCircuit(1 + n_target)
            u_gate.definition.append(Gate(f"U^{exp}", 1 + n_target, []), range(1 + n_target))
            
            # Apply controlled operation
            control_qubit = j
            target_qubits = list(range(n_count, n_count + n_target))
            
            # For simplicity, we'll use a placeholder for the actual controlled operation
            # In a real implementation, this would be the actual controlled unitary
            
        self.step_explanations.append(
            f"Step 3: Applied controlled modular multiplication gates for powers of {a}"
        )
        
        # Step 4: Apply inverse QFT
        qft_gate = QFTGate(n_count, do_swaps=False)
        qc.append(qft_gate.inverse(), range(n_count))
        self.step_explanations.append(
            f"Step 4: Applied inverse Quantum Fourier Transform to extract phase information"
        )
        
        # Step 5: Measure counting qubits
        qc.measure(range(n_count), range(n_count))
        self.step_explanations.append(
            f"Step 5: Measured counting qubits to obtain phase estimation results"
        )
        
        self.current_circuit = qc
        
        if show_circuit:
            self.display_circuit()
        
        # Run simulation
        tqc = transpile(qc, self.backend)
        job = self.backend.run(tqc, shots=shots)
        result = job.result()
        counts = result.get_counts()
        
        self.measurement_results = counts
        
        # Analyze results
        most_common = max(counts, key=counts.get)
        measured_int = int(most_common, 2)
        phase = measured_int / (2 ** n_count)
        
        denom, numer = self.continued_fraction_denominator(phase, N)
        
        return denom, qc, counts


In [ ]:
def display_circuit(self):
    """Display the quantum circuit with proper formatting"""
    if self.current_circuit is None:
        return
    
    fig, ax = plt.subplots(figsize=(15, 8))
    
    # Create a text representation of the circuit
    circuit_text = "Quantum Circuit for Order Finding:\n\n"
    circuit_text += "Counting Qubits: |+⟩⊗n → QFT⁻¹ → Measurement\n"
    circuit_text += "Target Qubits: |1⟩ → Controlled-U gates → |1⟩\n\n"
    circuit_text += "Key Components:\n"
    circuit_text += "• Hadamard gates create superposition\n"
    circuit_text += "• Controlled-U gates encode period information\n"
    circuit_text += "• Inverse QFT extracts period from phase\n"
    circuit_text += "• Measurement reveals period information"
    
    ax.text(0.1, 0.5, circuit_text, fontsize=12, verticalalignment='center',
            bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue", alpha=0.7))
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.axis('off')
    ax.set_title('Shor\'s Algorithm: Quantum Circuit Overview', fontsize=16, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

def display_probabilities(self):
    """Display measurement probabilities and analysis"""
    if self.measurement_results is None:
        return
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Plot histogram of measurement results
    bitstrings = list(self.measurement_results.keys())
    counts = list(self.measurement_results.values())
    
    # Sort by count for better visualization
    sorted_data = sorted(zip(bitstrings, counts), key=lambda x: x[1], reverse=True)
    top_10 = sorted_data[:10]  # Show top 10 results
    
    if top_10:
        bitstrings_top, counts_top = zip(*top_10)
        
        bars = ax1.bar(range(len(bitstrings_top)), counts_top, 
                        color='skyblue', edgecolor='navy', alpha=0.7)
        ax1.set_xlabel('Measurement Result (Binary)')
        ax1.set_ylabel('Count')
        ax1.set_title('Top 10 Measurement Results')
        ax1.set_xticks(range(len(bitstrings_top)))
        ax1.set_xticklabels(bitstrings_top, rotation=45)
        
        # Add count labels on bars
        for i, (bar, count) in enumerate(zip(bars, counts_top)):
            ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                    str(count), ha='center', va='bottom')
    
    # Analysis text
    total_shots = sum(self.measurement_results.values())
    most_common = max(self.measurement_results, key=self.measurement_results.get)
    most_common_count = self.measurement_results[most_common]
    probability = most_common_count / total_shots
    
    analysis_text = f"""Measurement Analysis:

Total Shots: {total_shots}
Most Common Result: {most_common}
Count: {most_common_count}
Probability: {probability:.3f}

Interpretation:
• The measurement result represents a phase estimate
• Higher probability indicates better phase estimation
• This phase is used to find the period of the function
• The period is crucial for factorization"""
        
    ax2.text(0.05, 0.95, analysis_text, transform=ax2.transAxes, fontsize=11,
            verticalalignment='top', bbox=dict(boxstyle="round,pad=0.5", 
            facecolor="lightgreen", alpha=0.7))
    ax2.set_xlim(0, 1)
    ax2.set_ylim(0, 1)
    ax2.axis('off')
    ax2.set_title('Probability Analysis', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

def display_step_by_step(self):
    """Display step-by-step calculation and explanation"""
    if not self.step_explanations:
        return
    
    fig, ax = plt.subplots(figsize=(14, 10))
    
    # Create a comprehensive explanation
    explanation_text = "Shor's Algorithm: Step-by-Step Process\n\n"
    
    for i, step in enumerate(self.step_explanations, 1):
        explanation_text += f"{i}. {step}\n\n"
    
    explanation_text += "\nMathematical Foundation:\n"
    explanation_text += "• We seek the period r such that a^r ≡ 1 (mod N)\n"
    explanation_text += "• If r is even and a^(r/2) ≢ ±1 (mod N), then:\n"
    explanation_text += "  gcd(a^(r/2) ± 1, N) gives non-trivial factors\n\n"
    explanation_text += "Quantum Advantage:\n"
    explanation_text += "• Classical algorithms: O(exp(√(log N log log N)))\n"
    explanation_text += "• Shor's algorithm: O((log N)³)\n"
    explanation_text += "• Exponential speedup for large numbers"
    
    ax.text(0.05, 0.95, explanation_text, transform=ax.transAxes, fontsize=11,
            verticalalignment='top', bbox=dict(boxstyle="round,pad=0.5", 
            facecolor="lightyellow", alpha=0.8))
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.axis('off')
    ax.set_title('Educational Guide: Understanding Shor\'s Algorithm', 
                fontsize=16, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

def shors_algorithm(self, N, n_count=8, shots=1024, tries=5, show_circuit=True):
    """Main Shor's algorithm implementation"""
    self.step_explanations = []
    
    # Check for trivial cases
    if N % 2 == 0:
        self.step_explanations.append(f"N = {N} is even, so 2 is a factor")
        return 2, N // 2
    
    for attempt in range(tries):
        a = np.random.randint(2, N)
        if gcd(a, N) != 1:
            g = gcd(a, N)
            self.step_explanations.append(f"Found gcd({a}, {N}) = {g} (trivial factor)")
            return g, N // g
        
        self.step_explanations.append(f"Attempt {attempt+1}: Trying a = {a}")
        r, qc, counts = self.find_order_qpe(a, N, n_count=n_count, shots=shots, show_circuit=show_circuit)
        
        if r is None:
            self.step_explanations.append("Failed to extract order from QPE result")
            continue
        
        self.step_explanations.append(f"Candidate order r = {r}")
        
        if r % 2 != 0:
            self.step_explanations.append("r is odd; trying another 'a'")
            continue
        
        ar2 = pow(a, r // 2, N)
        if ar2 == N - 1:
            self.step_explanations.append("a^(r/2) ≡ -1 (mod N); trying another 'a'")
            continue
        
        factor1 = gcd(ar2 - 1, N)
        factor2 = gcd(ar2 + 1, N)
        
        if factor1 in (1, N) or factor2 in (1, N):
            self.step_explanations.append("Found trivial factors; trying again")
            continue
        
        self.step_explanations.append(f"Success! Found factors: {factor1} and {factor2}")
        return factor1, factor2
    
    return None


In [ ]:
# Run the Shor's algorithm demonstration
print("🚀 Starting Shor's Algorithm Demo")
print("=" * 60)

# Run the main demonstration
shor_instance = run_shor_demo()

print("\n" + "=" * 60)
print("📚 Available Functions:")
print("• run_shor_demo() - Run the main algorithm")
print("• show_circuit_demo() - Show quantum circuit")
print("• show_probabilities_demo() - Show probability analysis")
print("• show_steps_demo() - Show educational content")
print("=" * 60)


In [ ]:
# Demonstrate additional features
print("🔬 Quantum Circuit Visualization Demo")
print("=" * 50)
show_circuit_demo()

print("\n📊 Probability Analysis Demo")
print("=" * 50)
show_probabilities_demo()

print("\n📚 Educational Content Demo")
print("=" * 50)
show_steps_demo()


In [ ]:
# Example: Run Shor's algorithm with different parameters
def run_custom_shor(N, n_count=6, shots=1024, tries=5):
    """Run Shor's algorithm with custom parameters"""
    shor = ShorAlgorithm()
    
    print(f"🚀 Running Shor's Algorithm to factor N = {N}")
    print(f"Parameters: n_count={n_count}, shots={shots}, tries={tries}")
    print("=" * 60)
    
    factors = shor.shors_algorithm(
        N=N,
        n_count=n_count,
        shots=shots,
        tries=tries,
        show_circuit=True
    )
    
    if factors is None:
        print(f"\n❌ Failed to find factors of {N} after {tries} attempts.")
        print("Try increasing the number of counting qubits or shots.")
    else:
        print(f"\n✅ Success! Found factors of {N}: {factors[0]} × {factors[1]} = {factors[0] * factors[1]}")
        print(f"\nVerification: {factors[0]} × {factors[1]} = {factors[0] * factors[1]}")
    
    return shor, factors

# Example runs with different numbers
print("🧮 Testing Shor's Algorithm on Different Numbers")
print("=" * 60)

# Test on 21
print("\n1. Testing N = 21:")
shor_21, factors_21 = run_custom_shor(21, n_count=6, shots=1024, tries=3)

# Test on 35
print("\n2. Testing N = 35:")
shor_35, factors_35 = run_custom_shor(35, n_count=6, shots=1024, tries=3)


In [ ]:
# Create the main GUI interface
def create_shor_gui():
    """Create an interactive GUI for Shor's algorithm"""
    
    # Create widgets
    N_widget = widgets.IntText(
        value=15,
        description='Number to factor (N):',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='300px')
    )
    
    n_count_widget = widgets.IntSlider(
        value=6,
        min=4,
        max=12,
        step=1,
        description='Counting qubits:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='300px')
    )
    
    shots_widget = widgets.IntSlider(
        value=1024,
        min=256,
        max=4096,
        step=256,
        description='Number of shots:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='300px')
    )
    
    tries_widget = widgets.IntSlider(
        value=5,
        min=1,
        max=10,
        step=1,
        description='Number of attempts:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='300px')
    )
    
    run_button = widgets.Button(
        description='Run Shor\'s Algorithm',
        button_style='success',
        layout=widgets.Layout(width='200px', height='40px')
    )
    
    circuit_button = widgets.Button(
        description='Show Circuit',
        button_style='info',
        layout=widgets.Layout(width='150px', height='40px')
    )
    
    prob_button = widgets.Button(
        description='Show Probabilities',
        button_style='warning',
        layout=widgets.Layout(width='150px', height='40px')
    )
    
    step_button = widgets.Button(
        description='Step-by-Step',
        button_style='primary',
        layout=widgets.Layout(width='150px', height='40px')
    )
    
    # Create output area
    output = widgets.Output()
    
    # Initialize Shor algorithm instance
    shor = ShorAlgorithm()
    
    def run_algorithm(b):
        with output:
            clear_output(wait=True)
            print(f"🚀 Running Shor's Algorithm to factor N = {N_widget.value}")
            print("=" * 50)
            
            factors = shor.shors_algorithm(
                N=N_widget.value,
                n_count=n_count_widget.value,
                shots=shots_widget.value,
                tries=tries_widget.value,
                show_circuit=True
            )
            
            if factors is None:
                print(f"\n❌ Failed to find factors of {N_widget.value} after {tries_widget.value} attempts.")
                print("Try increasing the number of counting qubits or shots.")
            else:
                print(f"\n✅ Success! Found factors of {N_widget.value}: {factors[0]} × {factors[1]} = {factors[0] * factors[1]}")
                print(f"\nVerification: {factors[0]} × {factors[1]} = {factors[0] * factors[1]}")
    
    def show_circuit(b):
        with output:
            clear_output(wait=True)
            print("🔬 Quantum Circuit Visualization")
            print("=" * 50)
            shor.display_circuit()
    
    def show_probabilities(b):
        with output:
            clear_output(wait=True)
            print("📊 Probability Analysis")
            print("=" * 50)
            shor.display_probabilities()
    
    def show_steps(b):
        with output:
            clear_output(wait=True)
            print("📚 Educational Content: Step-by-Step Guide")
            print("=" * 50)
            shor.display_step_by_step()
    
    # Connect buttons to functions
    run_button.on_click(run_algorithm)
    circuit_button.on_click(show_circuit)
    prob_button.on_click(show_probabilities)
    step_button.on_click(show_steps)
    
    # Create the main interface
    controls = widgets.VBox([
        widgets.HTML("<h2 style='color: #2E86AB; text-align: center;'>🔬 Shor's Algorithm Interactive Demo</h2>"),
        widgets.HTML("<p style='text-align: center; color: #666;'>Quantum Factorization with Educational Content</p>"),
        widgets.HBox([N_widget, n_count_widget]),
        widgets.HBox([shots_widget, tries_widget]),
        widgets.HBox([run_button, circuit_button, prob_button, step_button]),
        widgets.HTML("<hr style='margin: 20px 0;'>")
    ])
    
    # Create the complete interface
    interface = widgets.VBox([
        controls,
        output
    ])
    
    return interface, shor


In [ ]:
# Create and display the GUI
interface, shor_instance = create_shor_gui()
display(interface)


## Educational Content: Deep Dive into Quantum Concepts

### Quantum Fourier Transform (QFT) Explained

The Quantum Fourier Transform is a quantum algorithm that performs a Fourier transform on quantum amplitudes. It's a key component in many quantum algorithms, including Shor's algorithm.

#### Mathematical Definition
For an n-qubit system, the QFT transforms a quantum state |j⟩ as follows:

QFT|j⟩ = (1/√2ⁿ) Σ(k=0 to 2ⁿ-1) e^(2πijk/2ⁿ) |k⟩

#### Circuit Implementation
The QFT circuit consists of:
1. **Hadamard gates** on each qubit
2. **Controlled rotation gates** (R_k gates) with rotation angles 2π/2ᵏ
3. **Qubit swaps** to reverse the order

#### Role in Shor's Algorithm
- **Phase Extraction**: QFT converts phase information into computational basis states
- **Period Finding**: The inverse QFT extracts the period from the quantum state
- **Efficiency**: Provides exponential speedup over classical FFT

### Superposition and Entanglement

#### Quantum Superposition
Superposition allows quantum systems to exist in multiple states simultaneously:
- |ψ⟩ = α|0⟩ + β|1⟩ where |α|² + |β|² = 1
- Enables parallel computation on multiple states
- Collapses to a single state upon measurement

#### Entanglement
Entangled qubits have correlated properties:
- Bell states: |Φ⁺⟩ = (|00⟩ + |11⟩)/√2
- Measurement of one qubit determines the other
- Essential for quantum communication and computation

### Why Shor's Algorithm Works

1. **Period Finding**: The algorithm finds the period r of f(x) = aˣ mod N
2. **Quantum Speedup**: Uses quantum parallelism to evaluate f(x) for all x simultaneously
3. **Phase Estimation**: QPE extracts the period from the quantum state
4. **Classical Postprocessing**: Uses the period to find factors using number theory

### Complexity Analysis

| Algorithm | Time Complexity | Space Complexity |
|-----------|----------------|------------------|
| Classical (General Number Field Sieve) | O(exp((64/9)^(1/3) (log N)^(1/3) (log log N)^(2/3))) | O(log N) |
| Shor's Algorithm | O((log N)³) | O(log N) |

The exponential speedup makes Shor's algorithm a threat to RSA cryptography.


In [ ]:
# Additional educational visualizations
def create_educational_plots():
    """Create educational plots explaining quantum concepts"""
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
    
    # Plot 1: QFT Circuit Structure
    ax1.set_title('QFT Circuit Structure', fontsize=14, fontweight='bold')
    ax1.text(0.1, 0.8, 'QFT Circuit Components:', fontsize=12, fontweight='bold')
    ax1.text(0.1, 0.7, '1. Hadamard gates on each qubit', fontsize=10)
    ax1.text(0.1, 0.6, '2. Controlled rotation gates R_k', fontsize=10)
    ax1.text(0.1, 0.5, '3. Qubit swaps for correct order', fontsize=10)
    ax1.text(0.1, 0.3, 'Mathematical Formula:', fontsize=12, fontweight='bold')
    ax1.text(0.1, 0.2, 'QFT|j⟩ = (1/√2ⁿ) Σ e^(2πijk/2ⁿ)|k⟩', fontsize=10, 
             bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue"))
    ax1.set_xlim(0, 1)
    ax1.set_ylim(0, 1)
    ax1.axis('off')
    
    # Plot 2: Superposition Visualization
    ax2.set_title('Quantum Superposition', fontsize=14, fontweight='bold')
    theta = np.linspace(0, 2*np.pi, 100)
    x = np.cos(theta)
    y = np.sin(theta)
    ax2.plot(x, y, 'b-', linewidth=2, label='Bloch Sphere')
    ax2.arrow(0, 0, 1/np.sqrt(2), 1/np.sqrt(2), head_width=0.1, head_length=0.1, 
              fc='red', ec='red', linewidth=2)
    ax2.text(0.7, 0.7, '|+⟩ = (|0⟩ + |1⟩)/√2', fontsize=12, 
             bbox=dict(boxstyle="round,pad=0.3", facecolor="lightgreen"))
    ax2.set_xlim(-1.2, 1.2)
    ax2.set_ylim(-1.2, 1.2)
    ax2.set_aspect('equal')
    ax2.grid(True, alpha=0.3)
    ax2.legend()
    
    # Plot 3: Complexity Comparison
    ax3.set_title('Algorithm Complexity Comparison', fontsize=14, fontweight='bold')
    N_values = np.logspace(1, 4, 100)
    classical = np.exp(np.cbrt(64/9 * np.log(N_values) * (np.log(np.log(N_values)))**2))
    shor = (np.log(N_values))**3
    
    ax3.loglog(N_values, classical, 'r-', linewidth=2, label='Classical (GNFS)')
    ax3.loglog(N_values, shor, 'b-', linewidth=2, label="Shor's Algorithm")
    ax3.set_xlabel('Number of bits (log scale)')
    ax3.set_ylabel('Time complexity (log scale)')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # Plot 4: Shor's Algorithm Flow
    ax4.set_title('Shor\'s Algorithm Flow', fontsize=14, fontweight='bold')
    flow_text = """Shor's Algorithm Steps:

1. Input: Composite number N
2. Choose random a ∈ [2, N-1]
3. Check gcd(a, N) = 1
4. Find period r: a^r ≡ 1 (mod N)
5. If r is even and a^(r/2) ≢ ±1 (mod N):
   • factor1 = gcd(a^(r/2) - 1, N)
   • factor2 = gcd(a^(r/2) + 1, N)
6. Return factors or try different a

Quantum Advantage:
• Step 4 uses quantum phase estimation
• Exponential speedup over classical methods"""
    
    ax4.text(0.05, 0.95, flow_text, transform=ax4.transAxes, fontsize=10,
             verticalalignment='top', bbox=dict(boxstyle="round,pad=0.5", 
             facecolor="lightyellow", alpha=0.8))
    ax4.set_xlim(0, 1)
    ax4.set_ylim(0, 1)
    ax4.axis('off')
    
    plt.tight_layout()
    plt.show()

# Display educational plots
create_educational_plots()


## Advanced Features and Analysis

### Error Analysis and Success Probability

The success probability of Shor's algorithm depends on several factors:

1. **Phase Estimation Accuracy**: More counting qubits → higher accuracy
2. **Period Extraction**: Continued fraction algorithm success rate
3. **Random Choice of 'a'**: Probability of finding suitable period

### Quantum Circuit Optimization

Modern implementations use various optimizations:
- **Gate decomposition**: Breaking down complex unitaries
- **Error correction**: Handling quantum noise
- **Resource optimization**: Minimizing qubit and gate counts

### Real-World Considerations

1. **Noise**: Current quantum computers have limited coherence
2. **Scale**: Large numbers require many qubits
3. **Error Correction**: Needed for practical implementations
4. **Classical Postprocessing**: Still requires classical computation

### Future Directions

- **Fault-tolerant quantum computers**: Error-corrected implementations
- **Hybrid algorithms**: Combining quantum and classical methods
- **Cryptographic alternatives**: Post-quantum cryptography

## Conclusion

Shor's algorithm represents a landmark achievement in quantum computing, demonstrating the potential for exponential speedups over classical algorithms. While current implementations are limited by hardware constraints, the theoretical framework provides a foundation for future quantum algorithms and highlights the need for post-quantum cryptographic solutions.

The interactive demo above allows you to explore the algorithm with different parameters and understand the quantum mechanics behind this revolutionary algorithm.
